In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### tensorflow, kerasのVer.設定

In [2]:
!pip install --upgrade tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 30kB/s 
     |████████████████████████████████| 3.8MB 33.6MB/s 
     |████████████████████████████████| 512kB 49.8MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=ed344f3b6391332a7f605c7512967f7763ee09dba35cc5d6399571dd5dcdfdad
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [3]:
!pip install --upgrade keras==2.2.4

     |████████████████████████████████| 317kB 5.3MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


### モジュールのインポート

In [38]:
import sys
import os

sys.path.append('/content/drive/MyDrive/unet-master/')
from model import *
from data import *

#from keras.layers import Input
#import keras
#import tensorflow as tf

### test画像のファイル名のリネーム

（下方セルで読み込み処理がうまくいかないため、ファイル名のリネームで対応する。）

In [37]:
%cd '/content/drive/MyDrive/unet-master/TGS_Salt/competition_data/competition_data/test/images'

/content/drive/MyDrive/unet-master/TGS_Salt/competition_data/competition_data/test/images


In [ ]:
path = '/content/drive/MyDrive/unet-master/TGS_Salt/competition_data/competition_data/test/images'
file_list = os.listdir(path)

for num, file_name in enumerate(file_list):
    path1 = path + '/' + file_name
    path2 = path + '/' + str(num) + '.png'
    os.rename(path1, path2) 


### 【問題1】学習・推定

KaggleのTGS Salt Identification Challengeのデータセットを使用し、セグメンテーションを行う。

In [40]:
%cd /content/drive/MyDrive/unet-master

/content/drive/MyDrive/unet-master


In [14]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'TGS_Salt/competition_data/competition_data/train','images','masks',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=3,epochs=3,callbacks=[model_checkpoint])

/content/drive/MyDrive/unet-master/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


Epoch 1/3
Found 4000 images belonging to 1 classes.
Found 4000 images belonging to 1 classes.
3/3 [==============================] - 52s 17s/step - loss: 0.8474 - acc: 0.6368

Epoch 00001: loss improved from inf to 0.84739, saving model to unet_membrane.hdf5
Epoch 2/3
3/3 [==============================] - 47s 16s/step - loss: 0.6937 - acc: 0.7158

Epoch 00002: loss improved from 0.84739 to 0.69366, saving model to unet_membrane.hdf5
Epoch 3/3
3/3 [==============================] - 47s 16s/step - loss: 0.6936 - acc: 0.8986

Epoch 00003: loss improved from 0.69366 to 0.69356, saving model to unet_membrane.hdf5


In [36]:
testGene = testGenerator("/content/drive/MyDrive/unet-master/TGS_Salt/competition_data/competition_data/test/images")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test",results)

/content/drive/MyDrive/unet-master/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


30/30 [==============================] - 82s 3s/step


/content/drive/My Drive/unet-master/data.py:124: UserWarning: data/membrane/test/0_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/drive/My Drive/unet-master/data.py:124: UserWarning: data/membrane/test/1_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/drive/My Drive/unet-master/data.py:124: UserWarning: data/membrane/test/2_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/drive/My Drive/unet-master/data.py:124: UserWarning: data/membrane/test/3_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/drive/My Drive/unet-master/data.py:124: UserWarning: data/membrane/test/4_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/drive/My Drive/unet-master/data.py:124: UserWarning: data/membrane/test/5_predict.png is a low c

### 出力ファイルの確認

下記の通り推定データが出力・保存されている。

（今回の課題提出時は学習epochが少ないため、実際にはマスク画像は生成されていない）

In [41]:
%cd /content/drive/MyDrive/unet-master/data/membrane/test

/content/drive/MyDrive/unet-master/data/membrane/test


In [42]:
ls -a

0.png           15.png          20.png          26.png          4.png
0_predict.png   15_predict.png  20_predict.png  26_predict.png  4_predict.png
10.png          16.png          21.png          27.png          5.png
10_predict.png  16_predict.png  21_predict.png  27_predict.png  5_predict.png
11.png          17.png          22.png          28.png          6.png
11_predict.png  17_predict.png  22_predict.png  28_predict.png  6_predict.png
12.png          18.png          23.png          29.png          7.png
12_predict.png  18_predict.png  23_predict.png  29_predict.png  7_predict.png
13.png          19.png          24.png          2.png           8.png
13_predict.png  19_predict.png  24_predict.png  2_predict.png   8_predict.png
14.png          1.png           25.png          3.png           9.png
14_predict.png  1_predict.png   25_predict.png  3_predict.png   9_predict.png


### 【問題2】コードリーディング

・main.py No.7〜

　ImageDataGeneratorでデータ増強する際のパラメータを設定している。


・main.py No.14〜

　学習データ（X:元画像、y:マスク画像)はdata.pyのtrainGenerater関数で読み込んでいる。

　（本関数内で元画像、マスク画像共にkerasのImageDataGeneratorを使用して読み込んでいる。）
　　

・main.py No.16〜

　model.pyのunet関数を読み込んでいる。U-net論文と同様の構成となっている。unet関数はmodel型を戻す。

・main.py No.17、18

　学習の実施。ModelCheckpoint関数はKerasに含まれる関数で、過学習する前のモデルを保存、採用することができる。


・main.py No.20〜

　テスト画像の読み込み、推定の実施、推定画像の保存



